In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import numpy as np
import logging 
logging.basicConfig(level=logging.INFO)
import atlite
import crs 

In [2]:
filename="/groups/EXTREMES/cutouts/europe-2016-era5.nc"
ds = xr.open_dataset(filename)

In [7]:
ds

<xarray.Dataset> Size: 17GB
Dimensions:           (x: 189, y: 157, time: 8784)
Coordinates:
  * x                 (x) float64 2kB -12.0 -11.75 -11.5 ... 34.5 34.75 35.0
  * y                 (y) float64 1kB 33.0 33.25 33.5 33.75 ... 71.5 71.75 72.0
  * time              (time) datetime64[ns] 70kB 2016-01-01 ... 2016-12-31T23...
    lon               (x) float64 2kB ...
    lat               (y) float64 1kB ...
Data variables: (12/13)
    height            (y, x) float32 119kB ...
    wnd100m           (time, y, x) float32 1GB ...
    wnd_azimuth       (time, y, x) float32 1GB ...
    roughness         (time, y, x) float32 1GB ...
    influx_toa        (time, y, x) float32 1GB ...
    influx_direct     (time, y, x) float32 1GB ...
    ...                ...
    albedo            (time, y, x) float32 1GB ...
    solar_altitude    (time, y, x) float64 2GB ...
    solar_azimuth     (time, y, x) float64 2GB ...
    temperature       (time, y, x) float64 2GB ...
    soil temperature  (time, y, x) float64 2GB ...
    runoff            (time, y, x) float32 1GB ...
Attributes:
    module:             era5
    prepared_features:  ['influx', 'temperature', 'runoff', 'height', 'wind']
    chunksize_time:     100
    Conventions:        CF-1.6
    history:            2023-02-20 09:11:45 GMT by grib_to_netcdf-2.25.1: /op...

In [3]:
#cutout_path = "europe-2008-06-week-era5_new_wnd10_shear.nc"
cutout_path ="/groups/EXTREMES/cutouts/europe-2016-era5.nc"
 
# read in cutout instead of creating it
cutout = atlite.Cutout(path=cutout_path)

In [13]:
cutout.grid

,x,y,geometry
0,-12.00,33.0,"POLYGON ((-11.875 32.875, -11.875 33.125, -12...."
1,-11.75,33.0,"POLYGON ((-11.625 32.875, -11.625 33.125, -11...."
2,-11.50,33.0,"POLYGON ((-11.375 32.875, -11.375 33.125, -11...."
3,-11.25,33.0,"POLYGON ((-11.125 32.875, -11.125 33.125, -11...."
4,-11.00,33.0,"POLYGON ((-10.875 32.875, -10.875 33.125, -11...."
...,...,...,...
29668,34.00,72.0,"POLYGON ((34.125 71.875, 34.125 72.125, 33.875..."
29669,34.25,72.0,"POLYGON ((34.375 71.875, 34.375 72.125, 34.125..."
29670,34.50,72.0,"POLYGON ((34.625 71.875, 34.625 72.125, 34.375..."
29671,34.75,72.0,"POLYGON ((34.875 71.875, 34.875 72.125, 34.625..."


In [ ]:
print(cutout.grid.crs)
print(cutout.grid.head())

In [28]:
power_era5_2016=cutout.pv(
    panel='CSi', 
    orientation='latitude_optimal',
    tracking=None,
)

INFO:atlite.convert:Convert and aggregate 'pv'.


KeyboardInterrupt: 

In [29]:
power_era5_2016

<xarray.DataArray 'specific generation' (y: 157, x: 189)> Size: 237kB
array([[1665.117087  , 1670.82985761, 1675.62453298, ..., 1733.68983396,
        1724.15118419, 1713.5150573 ],
       [1654.5333218 , 1660.26905172, 1667.04610104, ..., 1732.17706531,
        1724.57313397, 1720.71018939],
       [1646.86677728, 1653.54548917, 1660.31920306, ..., 1732.02486402,
        1725.92714883, 1722.08074039],
       ...,
       [ 668.541382  ,  663.58420748,  660.24277091, ...,  717.41877686,
         716.21786177,  715.0961002 ],
       [ 672.07585724,  666.2621657 ,  661.25752118, ...,  705.85857593,
         704.28351569,  702.56793633],
       [ 673.47312207,  667.58092958,  662.66832355, ...,  691.90428341,
         690.56089424,  688.89504447]])
Coordinates:
  * y        (y) float64 1kB 33.0 33.25 33.5 33.75 ... 71.25 71.5 71.75 72.0
    lat      (y) float64 1kB 33.0 33.25 33.5 33.75 ... 71.25 71.5 71.75 72.0
  * x        (x) float64 2kB -12.0 -11.75 -11.5 -11.25 ... 34.25 34.5 34.75 35.0
    lon      (x) float64 2kB -12.0 -11.75 -11.5 -11.25 ... 34.25 34.5 34.75 35.0
Attributes:
    units:    kWh/kWp

In [30]:
filepath_model="/groups/FutureWind/SFCRAD/CanESM5/historical/r1i1p2f1/rsds_rsdsdiff_tas_2014.nc"
ds_model_2014 = xr.open_dataset(filepath_model)

In [45]:
ds_model_2014

<xarray.Dataset> Size: 20MB
Dimensions:   (time: 2920, lat: 20, lon: 28)
Coordinates:
  * lon       (lon) float32 112B -28.12 -25.31 -22.5 -19.69 ... 42.19 45.0 47.81
  * lat       (lat) float32 80B 20.93 23.72 26.51 29.3 ... 68.37 71.16 73.95
  * time      (time) object 23kB 2014-01-01 01:30:00 ... 2014-12-31 22:30:00
Data variables:
    rsds      (time, lat, lon) float32 7MB ...
    rsdsdiff  (time, lat, lon) float32 7MB ...
    tas       (time, lat, lon) float32 7MB ...
Attributes:
    data_source:    Processed data from CMIP6 runs
    experiment:     historical
    source:         CanESM5
    variant_label:  r1i1p2f1
    data_written:   12/03/2025 13:59

In [67]:
print(ds_model_2014.attrs)

{'data_source': 'Processed data from CMIP6 runs', 'experiment': 'historical', 'source': 'CanESM5', 'variant_label': 'r1i1p2f1', 'data_written': '12/03/2025 13:59'}


In [35]:
ds_era5_2016 = xr.open_dataset(cutout_path)

In [38]:
ds_era5_2016

<xarray.Dataset> Size: 17GB
Dimensions:           (x: 189, y: 157, time: 8784)
Coordinates:
  * x                 (x) float64 2kB -12.0 -11.75 -11.5 ... 34.5 34.75 35.0
  * y                 (y) float64 1kB 33.0 33.25 33.5 33.75 ... 71.5 71.75 72.0
  * time              (time) datetime64[ns] 70kB 2016-01-01 ... 2016-12-31T23...
    lon               (x) float64 2kB -12.0 -11.75 -11.5 ... 34.5 34.75 35.0
    lat               (y) float64 1kB 33.0 33.25 33.5 33.75 ... 71.5 71.75 72.0
Data variables: (12/13)
    height            (y, x) float32 119kB 0.8357 1.065 ... 0.2621 0.2621
    wnd100m           (time, y, x) float32 1GB 2.92 2.465 2.055 ... 15.38 15.32
    wnd_azimuth       (time, y, x) float32 1GB 0.428 0.3882 0.3743 ... 4.44 4.45
    roughness         (time, y, x) float32 1GB 2.426e-05 2.426e-05 ... 0.0003332
    influx_toa        (time, y, x) float32 1GB 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    influx_direct     (time, y, x) float32 1GB 3.472e-05 3.472e-05 ... 3.472e-05
    ...                ...
    albedo            (time, y, x) float32 1GB 1.0 1.0 1.0 1.0 ... 1.0 1.0 1.0
    solar_altitude    (time, y, x) float64 2GB -1.215 -1.218 ... -0.7083 -0.708
    solar_azimuth     (time, y, x) float64 2GB 5.13 5.137 ... 0.2419 0.2471
    temperature       (time, y, x) float64 2GB 290.7 290.6 290.6 ... 271.0 270.9
    soil temperature  (time, y, x) float64 2GB 292.6 292.6 292.6 ... 278.3 278.3
    runoff            (time, y, x) float32 1GB ...
Attributes:
    module:             era5
    prepared_features:  ['influx', 'temperature', 'runoff', 'height', 'wind']
    chunksize_time:     100
    Conventions:        CF-1.6
    history:            2023-02-20 09:11:45 GMT by grib_to_netcdf-2.25.1: /op...

In [39]:
ds_era5_2016_diff=ds_era5_2016['influx_diffuse'] 

In [48]:
from regridding_functions import regrid
regridder_era5_CanESM5=regrid(ds_era5_2016,ds_model_2014,'conservative')
era5_rsdsdiff=ds_era5_2016['influx_diffuse']
rsdsdiff_era5_model=regridder_era5_CanESM5(era5_rsdsdiff)

189 190
157 158
28 29
20 21


/home/s233224/anaconda3/envs/atlite_cmip6_2025/lib/python3.13/site-packages/xesmf/frontend.py:718: UserWarning: Using dimensions ('y', 'x') from data variable influx_diffuse as the horizontal dimensions for the regridding.
  warnings.warn(


In [49]:
rsdsdiff_era5_model


<xarray.DataArray (time: 8784, lat: 20, lon: 28)> Size: 20MB
array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
...
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]], dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 70kB 2016-01-01 ... 2016-12-31T23:00:00
  * lat      (lat) float32 80B 20.93 23.72 26.51 29.3 ... 68.37 71.16 73.95
  * lon      (lon) float32 112B -28.12 -25.31 -22.5 -19.69 ... 42.19 45.0 47.81
Attributes:
    regrid_method:  conservative

In [65]:
cutout = atlite.Cutout(path=filepath_model)

KeyError: None

In [51]:
print(ds_model_2014.attrs)

{'data_source': 'Processed data from CMIP6 runs', 'experiment': 'historical', 'source': 'CanESM5', 'variant_label': 'r1i1p2f1', 'data_written': '12/03/2025 13:59'}


In [53]:
print(ds_model_2014.coords)

Coordinates:
  * lon      (lon) float32 112B -28.12 -25.31 -22.5 -19.69 ... 42.19 45.0 47.81
  * lat      (lat) float32 80B 20.93 23.72 26.51 29.3 ... 68.37 71.16 73.95
  * time     (time) object 23kB 2014-01-01 01:30:00 ... 2014-12-31 22:30:00


In [57]:
ds_model_2014['CRS']='EPSG:4326'

In [58]:
print(ds_model_2014.attrs)

{'data_source': 'Processed data from CMIP6 runs', 'experiment': 'historical', 'source': 'CanESM5', 'variant_label': 'r1i1p2f1', 'data_written': '12/03/2025 13:59'}


In [63]:
ds_res=ds_era5_2016['time'].resample(time='3h',label="left", closed="right")